In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Reload all modules imported with %aimport
%load_ext autoreload
%autoreload 1

In [2]:
import pandas as pd
idx = pd.IndexSlice

from sklearn.pipeline import Pipeline, make_pipeline


In [3]:
%aimport trans.data
%aimport trans.gtrans

In [4]:
from trans.data import GetData as gd
from trans.gtrans import *

In [ ]:
sp500 = gd.save_sp500_tickers()

In [ ]:
len(sp500)

In [ ]:
all = gd.compile_data_all(['FB', 'AAPL', 'AMZN', 
                           'NFLX', 'GOOG'])

In [ ]:
all.loc['2017-12-22':'2017-12-27']


In [ ]:
gd.save_data(all, "test_all.pkl")

In [5]:
all = gd.load_data("test_all.pkl")

In [6]:
ids = gd.get_data_from_yahoo2([ 'SPY'])
ids_all = gd.compile_data_all(['SPY'])
gd.save_data(ids_all, "indexes_all.pkl")

Already have SPY
0


In [6]:
ids_all = gd.load_data("indexes_all.pkl")

## Rank: if there are n items, the sum of positions is sum from 1 to n, equals (n/2 * (1+n)).  The sum of the ranks obeys this too, with the caveat that multiple items having the same value have the same fractional rank, e.g., if there are 3 items, they will have the same rank r such tat 3*r = sum of positions of the three elements in the list

In [19]:
f = pd.DataFrame({ 'A': [ 20, 10, 20, 20], 'B': [20, 30, 10, 10]})
f
f.rank()
f.rank().astype(int)
f.apply(np.argsort)

,A,B
0,20,20
1,10,30
2,20,10
3,20,10


,A,B
0,3.0,3.0
1,1.0,4.0
2,3.0,1.5
3,3.0,1.5


,A,B
0,3,3
1,1,4
2,3,1
3,3,1


,A,B
0,1,2
1,0,3
2,2,0
3,3,1


In [8]:
from trans.gtrans import DataFrameFunctionTransformer 

pctTrans = DataFrameFunctionTransformer(func = lambda s: s.pct_change())
rankTrans = DataFrameFunctionTransformer(pd.Series.rank, axis=1)

In [64]:
from trans.gtrans import GenSelectAttrTransformer
from trans.gtrans import GenRenameAttrTransformer
from trans.gtrans import GenRankTransformer

pipe_1 = make_pipeline(GenSelectAttrTransformer(['Adj Close'] ), 
                       pctTrans,
                       GenRenameAttrTransformer(lambda col: col + ' pct')
                      )

pipe_2 = make_pipeline(GenSelectAttrTransformer(['Adj Close'] ), 
                       pctTrans, 
                       # rankTrans,
                       GenRankTransformer(),
                       GenRenameAttrTransformer(lambda col: col + ' rank')
                          )

featU = DataFrameFeatureUnion([ pipe_1, pipe_2 ])
u = featU.fit_transform(all.head())
u.head()
                    

,FB pct,AAPL pct,AMZN pct,NFLX pct,GOOG pct,FB rank,AAPL rank,AMZN rank,NFLX rank,GOOG rank
Date,,,,,,,,,,
2000-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-04,NaN,-0.084310,-0.083217,NaN,NaN,NaN,0.0,1.0,NaN,NaN
2000-01-05,NaN,0.014634,-0.148741,NaN,NaN,NaN,1.0,0.0,NaN,NaN
2000-01-06,NaN,-0.086538,-0.060036,NaN,NaN,NaN,0.0,1.0,NaN,NaN
2000-01-07,NaN,0.047368,0.061010,NaN,NaN,NaN,0.0,1.0,NaN,NaN


In [65]:
from trans.gtrans import GenDataFrameFeatureUnion

pipe_1 = make_pipeline(GenSelectAttrTransformer(['Adj Close'] ), 
                       pctTrans,
                       # GenRenameAttrTransformer(lambda col: col + ' pct')
                      )
pipe_2 = make_pipeline(GenSelectAttrTransformer(['Adj Close'] ), 
                       pctTrans,
                       # DataFrameFunctionTransformer(pd.Series.rank, axis=1)
                        GenRankTransformer(),
                       # GenRenameAttrTransformer(lambda col: col + ' rank')
                      )

featUn =  GenDataFrameFeatureUnion( [ ("Pct", pipe_1), ("Rank", pipe_2)])
un = featUn.fit_transform(all['2000-01-19':'2000-01-27'])
un.tail()

Pct                               Rank                    
            FB      AAPL      AMZN NFLX GOOG   FB AAPL AMZN NFLX GOOG
Date                                                                 
2000-01-21 NaN -0.019273 -0.041506  NaN  NaN  NaN  1.0  0.0  NaN  NaN
2000-01-24 NaN -0.045480  0.129909  NaN  NaN  NaN  0.0  1.0  NaN  NaN
2000-01-25 NaN  0.056471 -0.012478  NaN  NaN  NaN  1.0  0.0  NaN  NaN
2000-01-26 NaN -0.018374 -0.064079  NaN  NaN  NaN  1.0  0.0  NaN  NaN
2000-01-27 NaN -0.001702  0.032787  NaN  NaN  NaN  0.0  1.0  NaN  NaN

In [ ]:
u.columns
u.tail().loc[:, 'FB']